In [ ]:
import numpy as np
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from math import ceil, floor

In [ ]:
SAMPLE_NUMBER = 3
image_dir = Path.cwd()/"samples"/str(SAMPLE_NUMBER)
COORDS = np.load(image_dir/'points.npy')
IMAGES = [x for x in image_dir.iterdir() if x.suffix == ".jpg"]

In [ ]:
def solver(COORDS):
    '''
    COORDS : Nx2x2 array of coordinates
    x', y' = COORDS[i][0]
    x, y = COORDS[i][1]
    '''
    n, _, _ = COORDS.shape
    A = []
    B = []
    for i in range(n):
        x_, y_ = COORDS[i][0]
        x, y = COORDS[i][1]
        
        a = np.array([
            [x,y,1,0,0,0,-x*x_,-y*x_],
            [0,0,0,x,y,1,-x*y_,-y*y_]
        ])
        b = np.array([x_, y_])
        A.append(a)
        B.append(b)

    A = np.vstack(A)
    B = np.hstack(B)
    
    H = np.linalg.lstsq(A, B, rcond=None)[0]
    H = np.append(H, 1)
    H.resize((3,3))
    return H

In [ ]:
H = solver(COORDS)


# image 1
image1= cv2.imread(str(IMAGES[0]))
resized_img1 = np.zeros((4000, 6000, 3), dtype=np.uint8)
resized_img1[:image1.shape[0], :image1.shape[1], :] = image1
image1 = cv2.cvtColor(resized_img1, cv2.COLOR_BGR2RGB)
mask1 = np.any(image1 != [0, 0, 0], axis=2)
print(mask1.shape)
# image 2 warped
image = cv2.imread(str(IMAGES[1]))
output_size = (6000, 4000)
output = cv2.warpPerspective(image, H, output_size)
output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
mask2 = np.any(output != [0, 0, 0], axis=2)
print(mask2.shape)
# final image
final = np.zeros_like(output)
AND_MASK = np.logical_and(mask1, mask2)
final[AND_MASK] = cv2.addWeighted(image1, 0.5, output, 0.5, 0)[AND_MASK]
mask1_alone = np.logical_and(mask1, np.logical_not(mask2))
mask2_alone = np.logical_and(mask2, np.logical_not(mask1))
final[mask1_alone] = image1[mask1_alone]
final[mask2_alone] = output[mask2_alone]

In [ ]:
plt.imshow(final)
# save image
# cv2.imwrite("output.jpg", cv2.cvtColor(final, cv2.COLOR_RGB2BGR))

### Writing a `merge` function to combine all the images in a single canvas

In [ ]:
def get_bounding_box(image, H):
    # get all corners
    corners = np.array([
        [0, 0, 1],
        [image.shape[1], 0, 1],
        [0, image.shape[0], 1],
        [image.shape[1], image.shape[0], 1],
    ]).T

    # transform corners
    corners = H @ corners

    # normalize: x/w, y/w
    corners = corners / corners[2]

    # get bounding box
    min_x = floor(np.min(corners[0]))
    max_x = ceil(np.max(corners[0]))
    min_y = floor(np.min(corners[1]))
    max_y = ceil(np.max(corners[1]))
    
    return (min_x, max_x, min_y, max_y)

def get_translation_matrix(deltax, deltay):
    return np.array([
        [1, 0, deltax],
        [0, 1, deltay],
        [0, 0, 1]
    ], dtype=np.float32)

def merge(image_homographies_pairs):
    # homographies are with respect to the origin of the first image
    X, Y = float('inf'), float('inf')
    Xprime, Yprime = float('-inf'), float('-inf')
    for image, H in image_homographies_pairs:
        min_x, max_x, min_y, max_y = get_bounding_box(image, H)
        X = min(X, min_x)
        Y = min(Y, min_y)
        Xprime = max(Xprime, max_x)
        Yprime = max(Yprime, max_y)

    canvas_size = (Xprime - X, Yprime - Y)
    images = []
    for image, H in image_homographies_pairs:
        deltax = -X
        deltay = -Y
        translation_matrix = get_translation_matrix(deltax, deltay)
        warped = cv2.warpPerspective(image, translation_matrix @ H, canvas_size)
        images.append(warped)
    return images        

In [ ]:
SAMPLE_NUMBER = 1
image_dir = Path.cwd()/"samples"/str(SAMPLE_NUMBER)
COORDS = np.load(image_dir/'points.npy')
IMAGES = [x for x in sorted(image_dir.iterdir()) if x.suffix == ".jpg"]

In [ ]:
im1 = cv2.imread(str(IMAGES[0]))
im2 = cv2.imread(str(IMAGES[1]))
H = solver(COORDS)
ims = [(im1, np.eye(3)), (im2, H)]
im1, im2 = merge(ims)


In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(im2, cv2.COLOR_BGR2RGB))

### Blending with average

In [ ]:
im1_mask = np.any(im1 != [0, 0, 0], axis=2)
im2_mask = np.any(im2 != [0, 0, 0], axis=2)
common = np.logical_and(im1_mask, im2_mask)
common_image = cv2.addWeighted(im1, 0.5, im2, 0.5, 0)
final_image = im1 + im2
final_image[common] = common_image[common]

cv2.imwrite(str(image_dir/'output.jpg'), final_image)

In [ ]:
plt.imshow(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))

### Blending with linear gradient

In [ ]:
im1_mask = np.any(im1 != [0, 0, 0], axis=2)
im2_mask = np.any(im2 != [0, 0, 0], axis=2)
common = np.logical_and(im1_mask, im2_mask)
# leftmost in the common map
leftmost = (np.argmax(common, axis=1))
wow1 = np.max(common, axis=1)
wow1 = wow1.astype(bool)
leftmost = leftmost[wow1]
leftmost = min(leftmost)

rightmost = (np.argmax(common[:, ::-1], axis=1))
wow2 = np.max(common[:, ::-1], axis=1)
wow2 = wow2.astype(bool)
rightmost = rightmost[wow2]
rightmost = min(rightmost)
rightmost = im1.shape[1] - rightmost - 1

gradient = np.arange(im1.shape[1], dtype=np.float32)
gradient -= leftmost
gradient /= (rightmost - leftmost)
gradient = np.clip(gradient, 0, 1)

mask_gradient_right = common * gradient
mask_gradient_left = common - mask_gradient_right


# make the mask gradient R X C to R X C X 3
mask_gradient_right = np.repeat(mask_gradient_right[:, :, np.newaxis], 3, axis=2)
mask_gradient_left = np.repeat(mask_gradient_left[:, :, np.newaxis], 3, axis=2)

In [ ]:
final_image = im1 + im2
final_image[common] = (im1 * mask_gradient_left + im2 * mask_gradient_right)[common]
cv2.imwrite(str(image_dir/'output.jpg'), final_image)

In [ ]:
plt.imshow(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))

### Blending with laplacian pyramid

In [ ]:
im1_mask = np.any(im1 != [0, 0, 0], axis=2)
im2_mask = np.any(im2 != [0, 0, 0], axis=2)
mask = np.logical_and(im1_mask, im2_mask)
final_image = im1 + im2

In [ ]:
# def generate_laplacian_pyramid(image):
#     pyramid = [image]
#     for _ in range(3):
#         image = cv2.pyrDown(image)
#         pyramid.append(image)
#     laplacian_pyramid = [pyramid[i] - cv2.pyrUp(pyramid[i + 1], dstsize=(pyramid[i].shape[1], pyramid[i].shape[0])) for i in range(3)]
#     return laplacian_pyramid

# def blend_images(im1, im2):
#     im1_mask = np.any(im1 != [0, 0, 0], axis=2)
#     im2_mask = np.any(im2 != [0, 0, 0], axis=2)
#     mask = np.logical_and(im1_mask, im2_mask)

#     # Generate Laplacian pyramids
#     laplacian_pyramid_im1 = generate_laplacian_pyramid(im1)
#     laplacian_pyramid_im2 = generate_laplacian_pyramid(im2)

#     # Ensure masks have the same shape as the Laplacian pyramid levels
#     masks = [cv2.resize(mask.astype(np.uint8), (lap.shape[1], lap.shape[0])) for lap in laplacian_pyramid_im1]

#     # Blend the Laplacian pyramids
#     blended_pyramid = [m[:, :, np.newaxis] * lap1 + (1 - m[:, :, np.newaxis]) * lap2 for m, lap1, lap2 in zip(masks, laplacian_pyramid_im1, laplacian_pyramid_im2)]

#     # Reconstruct the final image
#     final_image = blended_pyramid[0]
#     for i in range(1, 4):
#         final_image = cv2.pyrUp(final_image, dstsize=(blended_pyramid[i].shape[1], blended_pyramid[i].shape[0]))
#         final_image += blended_pyramid[i]

#     return final_image


# final_result = blend_images(im1, im2)

# # # Display or save the result
# # cv2.imshow('Blended Image', final_result)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()
